## **CreditRiskClassification Model**


### Problem Defination:


In [6]:
# Import Data Manipulation Library
import numpy as np 
import pandas as pd 
# Import Data Visulization Library
import seaborn as sns
import matplotlib.pyplot as plt 
# Import FilterWarnings Library
import warnings 
warnings.filterwarnings('ignore')
#Import Logging 
import logging 
logging.basicConfig(level = logging.INFO,
                    filename = 'model.log',
                    filemode= 'w',
                    format= '%(asctime)s - %(levelname)s - %(message)s')
# Import Scikit- Learn Libaries 
from sklearn.preprocessing import MinMaxScaler,RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report , confusion_matrix